In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
import utils
from utils import util_functions

In [2]:
df_path = 'dataset/state_media_on_social_media_platforms.xlsx'
# loading the dataset
df = pd.read_excel(df_path, index_col='Name (English)')
# Load JSON data from file
with open('assets/recognized_countries.json', 'r') as file:
    data = json.load(file)
recognized_countries = data

with open('assets/equivalent_countries.json', 'r') as file:
    data = json.load(file)
equivalent_countries_dict = data

In [3]:
country_focus_count_dict = util_functions.find_country_focus_count_dictionary(df, recognized_countries, equivalent_countries_dict)

c:\src\jupyter_codes\foreign-affairs\utils\util_functions.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country_list = df['focus group'][idx]


In [4]:
df = util_functions.data_preprocessor(df)
df.head(1)

Region of Focus Language   Entity owner (English)  \
Name (English)                                                               
Yang Xinmeng (Abby Yang)     Anglosphere  English  China Media Group (CMG)   

                               Parent entity (English) X (Twitter) handle  \
Name (English)                                                              
Yang Xinmeng (Abby Yang)  Central Publicity Department       _bubblyabby_   

                                           X (Twitter) URL  \
Name (English)                                               
Yang Xinmeng (Abby Yang)  https://twitter.com/_bubblyabby_   

                          X (Twitter) Follower #            Facebook page  \
Name (English)                                                              
Yang Xinmeng (Abby Yang)                  1678.0  itsAbby-103043374799622   

                                                              Facebook URL  \
Name (English)                                                               
Yang Xinmeng (Abby Yang)  https://www.facebook.com/itsAbby-103043374799622   

                          Facebook Follower #  ... Threads account  \
Name (English)                                 ...                   
Yang Xinmeng (Abby Yang)            1387432.0  ...    _bubblyabby_   

                                                    Threads URL  \
Name (English)                                                    
Yang Xinmeng (Abby Yang)  https://www.threads.net/@_bubblyabby_   

                          Threads Follower # YouTube account  \
Name (English)                                                 
Yang Xinmeng (Abby Yang)               197.0         itsAbby   

                                              YouTube URL  \
Name (English)                                              
Yang Xinmeng (Abby Yang)  https://www.youtube.com/itsAbby   

                          YouTube Subscriber # TikTok account  \
Name (English)                                                  
Yang Xinmeng (Abby Yang)                4680.0   _bubblyabby_   

                                                    TikTok URL  \
Name (English)                                                   
Yang Xinmeng (Abby Yang)  https://www.tiktok.com/@_bubblyabby_   

                          TikTok Subscriber #  \
Name (English)                                  
Yang Xinmeng (Abby Yang)                660.0   

                                                                focus group  
Name (English)                                                               
Yang Xinmeng (Abby Yang)  [Australia, Canada, New Zealand, United Kingdo...  

[1 rows x 23 columns]

In [5]:
language_usage_series = df['Language'].value_counts()

In [6]:
twitter_follower_series = df['X (Twitter) Follower #']
twitter_follower_series = twitter_follower_series.dropna()
twitter_follower_series.sort_values(ascending=False, inplace=True)
twitter_follower_series

Name (English)
CGTN                        13000000.0
Xinhua News Agency          11900000.0
People's Daily               6600000.0
China Daily                  4100000.0
Hua Chunying                 2200000.0
                               ...    
Ji Yuqiao                          3.0
Zhang Lin (Melody Zhang)           3.0
Love Jiaxing                       2.0
CRI Mongolian                      0.0
CGTN Inheritors                    0.0
Name: X (Twitter) Follower #, Length: 573, dtype: float64

In [35]:
column_list = ['Name (English)', 'Entity owner (English)', 'Parent entity (English)']

unique_parents = df['Parent entity (English)'].unique()
unique_owners = df['Entity owner (English)'].unique()
unique_names = df.index
df['Account Name'] = df.index

edges = []
nodes = []
node_colors = dict()

for owner in unique_owners:
    # owner = 'China Media Group (CMG)'
    if owner not in nodes:
        nodes.append(owner)
    to_node = owner
    node_colors[to_node] = 'blue'
    from_node = df[df['Entity owner (English)'] == owner]['Parent entity (English)'].unique()[0]
    node_colors[from_node] = 'red'
    if from_node not in nodes:
        nodes.append(from_node)
    # to nodes and from nodes are vise versa :)
    edges.append((from_node, to_node))

for account in unique_names:
    # owner = 'China Media Group (CMG)'
    if account not in nodes:
        nodes.append(account)
    to_node = account
    node_colors[to_node] = 'green'
    from_node = df[df['Account Name'] == account]['Entity owner (English)'].unique()[0]
    # node_colors[from_node] = 'blue'
    if from_node not in nodes:
        nodes.append(from_node)
    # to nodes and from nodes are vise versa :)
    edges.append((from_node, to_node))
    
graph_dict = {"nodes": nodes, "edges": edges, 'node_colors': node_colors}
util_functions.save_country_focus_count_dict_file(graph_dict, "findings/account_owner_parent.json")

In [9]:
df.columns

Index(['Region of Focus', 'Language', 'Entity owner (English)',
       'Parent entity (English)', 'X (Twitter) handle', 'X (Twitter) URL',
       'X (Twitter) Follower #', 'Facebook page', 'Facebook URL',
       'Facebook Follower #', 'Instragram page', 'Instagram URL',
       'Instagram Follower #', 'Threads account', 'Threads URL',
       'Threads Follower #', 'YouTube account', 'YouTube URL',
       'YouTube Subscriber #', 'TikTok account', 'TikTok URL',
       'TikTok Subscriber #', 'focus group'],
      dtype='object')

In [ ]:
['X (Twitter) Follower #', 'Facebook Follower #', 'YouTube Subscriber #', 'Instagram Follower #', 'Threads Follower #', 'TikTok Subscriber #']

In [20]:
type(twitter_follower_series)

pandas.core.series.Series